In [ ]:
import numpy as np
import glob
import fitting_output as fit_out
import lattice_parameters as lp

## Setup of fitting parameters for your structure (Example: LiNiO2)
Params is the dictionary (of dictionaries) which contains the main information relating to the system and potentials. There are 5 sub dictionaries: core_shell, charges, masses, potentials, and cs_springs.

**core_shell**: The keys are each atom type within the structure, with the relating value a boolean expression stating if that atom type is core-shell or not i.e. True = core-shell, False = rigid ion.

**charges**: The keys are again each atom type within the structure. The relating value is either a float representation of the atomic charge (for rigid ion atoms) or a sub dictionary where the sub keys are 'core' and 'shell', with the relating sub values being a float representation of the charge. Note: if you are fitting the charge separation (dq), the formal charge should be on the core and 0.0 charge on the shell.

**masses**: Similar to charges, the keys are each atom type within the structure, with the values either a float representation of the atomic mass, or a sub directory with the sub keys 'core' and 'shell' and sub values a float representation of the mass on each (summing to the atomic mass). Mass can not be fitted, and there is no definative way of splitting the mass, however convention suggests having 10 % mass on the shell.

**potentials**: The keys are atom label pairs separated by a dash (str), example: 'Li-O'. The values are a list of the buckingham potential parameters, i.e. [a, rho, c], where each parameter is a float.

**cs_springs**: The keys are agian atom label pairs separated by a dash (str), example: 'O-O'. This basically denotes the spring is between 'O' core and 'O' shell. The values are a list of the spring contants, k1 and k2, as floats. Commonly k2 is set to 0.0.

In [ ]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0,
                           'shell': 0.0}} 
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['potentials'] = {'Li-O': [663.111, 0.119, 0.0],
                        'Ni-O': [1393.540, 0.218, 0.000],
                        'O-O': [25804.807, 0.284, 0.0]}
params['cs_springs'] = {'O-O' : [20.0, 0.0]}

## Directory set up

Structures is a required input. This designate how many structures are in the training set (saved in a sub directory named 'training_set' in both the 'poscars' and 'outcars' directories). The lattice parameters are calculated for all structures in the training set.

The 'directory' is set up to read the potental from a results directory, containing sub directories with 'potentials.json' files. This can be changed to your results directory.

'lp_directory' is the lattice parameters sub directory which is created to save the lattice parameters of each structure (for each potential), differences from the reference lattice parameters, and relating plots.

In [ ]:
structures = 15 #Total number of structures in the training set
directory = 'results/formal+coreshell'

lp_directory = 'lattice_parameters'
output_directory = fit_out.create_directory(directory, lp_directory)

## Reference data (a, b, c, volume)
Here is where you state the reference you want to compare your lattice parameters (a,b,c) and volumes to. This needs to be the same scale as (or adjusted to) the structure cell size.

In [ ]:
labels = ['a ($\AA$)', 'b ($\AA$)', 'c ($\AA$)', 'volume ($\AA^2$)']
ref_DFT = [10.916, 10.061, 14.053, 1543.293] #(2.7289340495999999*4, (5.0304288864000002*2), 14.053= (14.0527168296999996*1), volume is a*b*c =1543.2928215270413)
ref_DFT = np.array([ref_DFT[0]*2,ref_DFT[1]*2,ref_DFT[2]*2,ref_DFT[3]*8])

## Running MD with each potential and collecting the lattice parameters for each structure

The lattice parameters are calculated by running a shell minimisation (if core-shell) followed by a cell relaxation. The setting for this, such as time step and minimiser settings are set in the 'get_lattice_params' function in 'fitting_code.py'.

For each potential in the subdirectories of the working directory, the potential is taken and used to relax every structure in the training set (independently), returning the resulting lattice parameters for each. These are saved in the 'lattice_parameters' subdirectory named with the structures in the potential fit followed by 'lattice_values.dat'.

The percentage difference is also calculated between the lattice parameters from the relaxation and the reference lattice parameters. These are also saved in the 'lattice_parameters' subdirectory named with the structures in the potential fit followed by 'lattice_diffs.dat'.

In [ ]:
percent_difference = lp.run_relaxation(structures, directory, output_directory, params, labels, ref_DFT, supercell=[2,2,2])       

## Plot lattice parameters (multiplot)

Firstly, for each lattice parameter set, the relaxed lattice parameters, of all training set structures,  from the potential are read in from the sub directories within the 'output_directory' and stored in a numpy array.

The lattice parameters (a, b, c, and volume) are then plotted and saved in the output directory, i.e. the 'lattice_parameters' directory.

In [ ]:
for potential_file in sorted(glob.glob('{}/*/potentials.json'.format(directory))):
    pot_structures = potential_file.replace('/potentials.json', '').replace('{}/'.format(directory),'')
    calculated_parameters = np.loadtxt('{}/{}_lattice_values.dat'.format(output_directory, pot_structures))
    lp.plot_lattice_params(labels, calculated_parameters, ref_DFT, output_directory, pot_structures, save=True)

## Plot lattice parameters with distribution curves (individual plots)

Firstly, for each lattice parameter set, the relaxed lattice parameters, of all training set structures, from the potential are read in from the sub directories within the 'output_directory' and stored in a numpy array.

The lattice parameters (a, b, c, and volume) are then plotted with a y-axis distribution and saved in the output directory, i.e. the 'lattice_parameters' directory, as individual plots.

In [ ]:
for potential_file in sorted(glob.glob('{}/*/potentials.json'.format(directory))):
    pot_structures = potential_file.replace('/potentials.json', '').replace('{}/'.format(directory),'')
    calculated_parameters = np.loadtxt('{}/{}_lattice_values.dat'.format(output_directory, pot_structures))
    lp.plot_lattice_params_with_distributions(labels, calculated_parameters, ref_DFT, output_directory, pot_structures, save=True)

## not in use

suppose to print all lattice parameters in one image with the distributions but the distributions overlay the y-axis lables. I haven't worked out how to fixe this yet

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib import gridspec

# labels = ['a ($\AA$)', 'b ($\AA$)', 'c ($\AA$)', 'volume ($\AA^2$)']
# fig, axs = plt.subplots(1, 8, figsize=(20,8), sharey=True)
# plt.subplots_adjust(wspace=0.0)
# gs = gridspec.GridSpec(1, 8, width_ratios=[1, 0.5, 1, 0.5, 1, 0.5, 1, 0.5])

# for i, j in enumerate(range(0,8,2)):
#     y = calculated_parameters.T[i]
#     x = np.random.rand(len(y))
#     ylims = y_limits(ref_DFT[i], y)  
#     axs[j] = scatter_plot(x,y,ref_DFT[i],labels[i],gs[j])
#     axs[j+1] = distribution_plot(y, gs[j+1])
#     ylims = y_limits(ref_DFT[i], y)
#     axs[j].set_ylim(ylims)
#     axs[j+1].set(ylim =ylims)
# #     plt.savefig('plots/3_structure_lattice_comparison.png',dpi=500, bbox_inches = "tight")
# plt.show()